In [70]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import ConfigurableField

from neo4j import GraphDatabase
from langchain_community.vectorstores import Neo4jVector
from langchain_community.graphs import Neo4jGraph

import pickle
import os

In [67]:
OPENAI_API_KEY = 'sk-XwwGmbuN8PZ2IX5CAkejMXIrPpsF4c435PEGWu0NIbT3BlbkFJAW1uxKGNBG1DyPZR43e06JGPrCvPrWKYmbDlTxzMoA'

In [68]:
NEO4J_URI="neo4j+s://0ca84f80.databases.neo4j.io"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="IzA_n5m2noHPKDTX5F3AlaosozzHkAxHghM71n69SpI"

In [71]:
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["NEO4J_URI"] = NEO4J_URI
os.environ["NEO4J_USERNAME"] = NEO4J_USERNAME
os.environ["NEO4J_PASSWORD"] = NEO4J_PASSWORD

In [52]:

# Load the cleaned_documents list from the file
with open('final_pdf_pages.pkl', 'rb') as file:
    docs = pickle.load(file)

print("Documents loaded successfully!")


Documents loaded successfully!


In [54]:
# docs[0]

In [57]:
len(docs)

220

In [55]:
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    separators=['\n'],
    chunk_size=2024,
    chunk_overlap=204,
    length_function=len,
)

In [56]:
data = text_splitter.split_documents(docs)

In [58]:
len(data)

615

In [61]:
# print(data[0].page_content)

In [63]:
type(data[0])

langchain_core.documents.base.Document

In [64]:
# Saving Chunked data
import pickle

# Save the cleaned_documents list to a file
with open('chunked_documents.pkl', 'wb') as file:
    pickle.dump(cleaned_documents, file)

print("Documents saved successfully!")

Documents saved successfully!


In [78]:
from langchain_experimental.graph_transformers import LLMGraphTransformer

In [ ]:
llm_transformer = LLMGraphTransformer(llm=llm)

In [80]:
from langchain_openai import ChatOpenAI

PydanticUserError: The `__modify_schema__` method is not supported in Pydantic v2. Use `__get_pydantic_json_schema__` instead in class `SecretStr`.

For further information visit https://errors.pydantic.dev/2.9/u/custom-json-schema

In [73]:
graph = Neo4jGraph()